In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import anndata as ad
import numpy as np
import pandas as pd

from scaleflow.data import DataManager
from scaleflow.data._anndata_location import AnnDataLocation


def adata_test():
    drugs = ["control", "drug_A", "drug_B"]
    genes = ["control", "gene_A", "gene_B"]
    cell_lines = ["cell_line_A", "cell_line_B"]
    batches = ["batch_1", "batch_2", "batch_3"]
    plates = ["plate_1", "plate_2", "plate_3"]
    days = ["day_1", "day_2", "day_3"]
    doses = [1.0, 10.0, 100.0]

    rows = []
    for drug in drugs:
        for gene in genes:
            for cell_line in cell_lines:
                for batch in batches:
                    for plate in plates:
                        for day in days:
                            if drug != "control":
                                for dose in doses:
                                    rows.append(
                                        {
                                            "drug": drug,
                                            "gene": gene,
                                            "cell_line": cell_line,
                                            "batch": batch,
                                            "plate": plate,
                                            "day": day,
                                            "dose": dose,
                                            "control": False,
                                        }
                                    )
                            else:
                                rows.append(
                                    {
                                        "drug": drug,
                                        "gene": gene,
                                        "cell_line": cell_line,
                                        "batch": batch,
                                        "plate": plate,
                                        "day": day,
                                        "dose": 0.0,
                                        "control": gene == "control" and drug == "control",
                                    }
                                )

    n_obs = len(rows)
    n_vars = 20
    n_pca = 10

    obs = pd.DataFrame(rows)

    # Convert to categorical
    for col in ["cell_line", "drug", "gene", "batch", "plate", "day"]:
        obs[col] = obs[col].astype("category")

    # Simple X matrix (not really used in tests, just needs to exist)
    X = np.random.randn(n_obs, n_vars).astype(np.float32)

    # X_pca: Put cell index at position [idx, 0] for easy tracing
    X_pca = np.zeros((n_obs, n_pca), dtype=np.float32)
    for i in range(n_obs):
        X_pca[i, 0] = float(i)  # Cell 0 has value 0, cell 1 has value 1, etc.

    # Create AnnData
    adata = ad.AnnData(X=X, obs=obs)
    adata.obsm["X_pca"] = X_pca

    # Simple embeddings
    adata.uns["cell_line_embeddings"] = {
        "cell_line_A": np.array([1.0, 0.0], dtype=np.float32),
        "cell_line_B": np.array([0.0, 1.0], dtype=np.float32),
    }

    adata.uns["drug_embeddings"] = {
        "drug_A": np.array([1.0, 0.0, 0.0], dtype=np.float32),
        "drug_B": np.array([0.0, 1.0, 0.0], dtype=np.float32),
        "control": np.array([0.0, 0.0, 0.0], dtype=np.float32),
    }

    adata.uns["gene_embeddings"] = {
        "gene_A": np.array([1.0, 0.0], dtype=np.float32),
        "gene_B": np.array([0.0, 1.0], dtype=np.float32),
        "control": np.array([0.0, 0.0], dtype=np.float32),
    }

    return adata

/Users/selman.ozleyen/mambaforge/envs/scaleflow/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
adata = adata_test()

/Users/selman.ozleyen/mambaforge/envs/scaleflow/lib/python3.12/functools.py:912: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)


In [6]:
"""Test that prepare_data works and returns correct structure."""
adl = AnnDataLocation()

dm = DataManager(
    dist_flag_key="control",
    src_dist_keys=["cell_line"],
    tgt_dist_keys=["drug", "gene"],
    rep_keys={
        "cell_line": "cell_line_embeddings",
        "drug": "drug_embeddings",
        "gene": "gene_embeddings",
    },
    data_location=adl.obsm["X_pca"],
)

In [7]:
gd = dm.prepare_data(adata,verbose=False)

In [9]:
from scaleflow.data._data_splitter import AnnotationSplitter


In [20]:
splitter = AnnotationSplitter(
    annotation=gd.annotation,
    holdout_combinations=False,
    split_by=["drug"],
    split_key="split",
    force_training_values={},
    ratios=[0.4, 0.3, 0.3],
    random_state=42,
)

In [21]:
splitter.show();

Split Summary

Total unique (src_dist_idx, tgt_dist_idx) combinations: 16

Split distribution (unique combinations):
----------------------------------------
  train     :      4 ( 25.0%)
  val       :      6 ( 37.5%)
  test      :      6 ( 37.5%)

Total rows in dataframe: 16

Split distribution (all rows):
----------------------------------------
  train     :      4 ( 25.0%)
  val       :      6 ( 37.5%)
  test      :      6 ( 37.5%)

Per-split breakdown:
----------------------------------------
  train     :    2 src dists,    4 tgt dists
  val       :    2 src dists,    6 tgt dists
  test      :    2 src dists,    6 tgt dists
